In [1]:
import sys
sys.path.append('..')
import pickle
from kss import split_chunks
from kss import split_sentences
import json
import re
from tqdm.auto import tqdm

In [14]:
with open('/opt/ml/input/data/data/wikipedia_documents.json') as f:
    wiki_docs = json.load(f)
wiki_docs = list(dict.fromkeys([v['text'] for v in wiki_docs.values()]))

In [15]:
wiki_docs = [re.sub(r'\xa0',' ', cxt) for cxt in wiki_docs]
wiki_docs = [re.sub(r'\\n','\n', cxt) for cxt in wiki_docs]
wiki_docs = [re.sub(r'[^\w\s\'\".,?!]',' ', cxt) for cxt in wiki_docs]
wiki_docs = [re.sub(r'( )+',' ', cxt) for cxt in wiki_docs]

In [17]:
cxt_list = []
for cxt in tqdm(wiki_docs):
    cxt_list.append(split_sentences(cxt))

process_list = []
for cxt in tqdm(cxt_list):
    tmp = []
    for sentence in cxt:
        if re.search(r'[ㄱ-ㅎㅏ-ㅣ가-힣]', sentence) != None:
            tmp.append(sentence)

with open('/opt/ml/input/data/data/processed_wiki_list.bin', 'wb') as f:
    pickle.dump(process_list,f)

In [3]:
with open('/opt/ml/input/data/data/processed_wiki_list.bin', 'rb') as f:
    process_list = pickle.load(f)

split_wiki_list_1024 = [] #45234 없애버림
for cxt in tqdm(process_list[:45234]):
    if cxt == '':
        continue
    for chunk in split_chunks(cxt, max_length=1024, overlap=True):
        split_wiki_list_1024.append(chunk.text)
        
for cxt in tqdm(process_list[45235:]):
    if cxt == '':
        continue
    for chunk in split_chunks(cxt, max_length=1024, overlap=True):
        split_wiki_list_1024.append(chunk.text)

with open('/opt/ml/input/data/data/split_wiki_list_1024.bin', 'wb') as f:
    pickle.dump(split_wiki_list_1024,f)